# apenas 12 variaveis - todos vazamentos
## timesteps = 2

In [1]:
import pandas as pd
import xlrd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

### Function create time series

In [2]:
#n_in: time steps
def series_to_supervised(data, n_in, n_out=1):
    df = pd.DataFrame(data)
    cols = list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        # put it all together
        agg = pd.concat(cols, axis=1)
        # drop rows with NaN values
        agg.dropna(inplace=True)
    return agg.values

In [3]:
# testing function
df = pd.DataFrame({"Col1": [10, 20, 15, 30, 45],
                   "Col2": [13, 23, 18, 33, 48],
                   "Col3": [17, 27, 22, 37, 52],
                  "Col4":[1,2,3,4,5]},
                  index=pd.date_range("2020-01-01", "2020-01-05"))
display(df)
series_to_supervised(df,2)

,Col1,Col2,Col3,Col4
2020-01-01,10,13,17,1
2020-01-02,20,23,27,2
2020-01-03,15,18,22,3
2020-01-04,30,33,37,4
2020-01-05,45,48,52,5


array([[10., 13., 17.,  1., 20., 23., 27.,  2., 15., 18., 22.,  3.],
       [20., 23., 27.,  2., 15., 18., 22.,  3., 30., 33., 37.,  4.],
       [15., 18., 22.,  3., 30., 33., 37.,  4., 45., 48., 52.,  5.]])

In [4]:
variables = ["TEI","TEO","TCI","TCO","PRE","PRC","T_suc","TR_dis","Amps","kW","TO_feed","PO_feed"]
print(len(variables))

12


In [5]:
sheet = "Complete Data Set"
path_to_xls = "F:/Talles/Doutorado/database/1043-RP/FDD Data/Refrigerant leak/rl10.xls"
leaking_10 = pd.read_excel(path_to_xls,sheet)
leaking_10 = leaking_10.loc[:, variables]
path_to_xls = "F:/Talles/Doutorado/database/1043-RP/FDD Data/Refrigerant leak/rl20.xls"
leaking_20 = pd.read_excel(path_to_xls,sheet)
leaking_20 = leaking_20.loc[:, variables]
path_to_xls = "F:/Talles/Doutorado/database/1043-RP/FDD Data/Refrigerant leak/rl30.xls"
leaking_30 = pd.read_excel(path_to_xls,sheet)
leaking_30 = leaking_30.loc[:, variables]
path_to_xls = "F:/Talles/Doutorado/database/1043-RP/FDD Data/Refrigerant leak/rl40.xls"
leaking_40 = pd.read_excel(path_to_xls,sheet)
leaking_40 = leaking_40.loc[:, variables]
path_to_xls = "F:/Talles/Doutorado/database/1043-RP/FDD Data/Benchmark Tests/normal r.xls"
normal = pd.read_excel(path_to_xls,sheet)
normal = normal.loc[:, variables]
print("shapes -- leaking10 = {}, leaking 20 = {}, leaking30 = {}, leaking40 = {}, normal = {}".format(leaking_10.shape,
                                                                                                      leaking_20.shape,
                                                                                                      leaking_30.shape,
                                                                                                      leaking_40.shape,
                                                                                                      normal.shape))

shapes -- leaking10 = (5191, 12), leaking 20 = (5191, 12), leaking30 = (5191, 12), leaking40 = (5191, 12), normal = (5191, 12)


In [6]:
timesteps = 2
leaking_10 = series_to_supervised(leaking_10,timesteps)
leaking_20 = series_to_supervised(leaking_20,timesteps)
leaking_30 = series_to_supervised(leaking_30,timesteps)
leaking_40 = series_to_supervised(leaking_40,timesteps)
normal = series_to_supervised(normal,timesteps)
print("shapes -- leaking10 = {}, leaking 20 = {}, leaking30 = {}, leaking40 = {}, normal = {}".format(leaking_10.shape,
                                                                                                      leaking_20.shape,
                                                                                                      leaking_30.shape,
                                                                                                      leaking_40.shape,
                                                                                                      normal.shape))

shapes -- leaking10 = (5189, 36), leaking 20 = (5189, 36), leaking30 = (5189, 36), leaking40 = (5189, 36), normal = (5189, 36)


In [7]:
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler

In [8]:
normal = np.c_[normal,np.zeros(len(normal))]
leaking_10 = np.c_[leaking_10,np.ones(len(leaking_10))]
leaking_20 = np.c_[leaking_20,np.ones(len(leaking_20))*2]
leaking_30 = np.c_[leaking_30,np.ones(len(leaking_30))*3]
leaking_40 = np.c_[leaking_40,np.ones(len(leaking_40))*4]
print("shapes -- leaking10 = {}, leaking 20 = {}, leaking30 = {}, leaking40 = {}, normal = {}".format(leaking_10.shape,
                                                                                                      leaking_20.shape,
                                                                                                      leaking_30.shape,
                                                                                                      leaking_40.shape,
                                                                                                      normal.shape))

shapes -- leaking10 = (5189, 37), leaking 20 = (5189, 37), leaking30 = (5189, 37), leaking40 = (5189, 37), normal = (5189, 37)


# Prepare dataset

In [9]:
data = np.vstack((normal,leaking_10))
data = np.vstack((data,leaking_20))
data = np.vstack((data,leaking_30))
data = np.vstack((data,leaking_40))
data.shape

(25945, 37)

In [10]:
x,y = (data[:,:-1],data[:,-1])
y = np.expand_dims(y, axis=1)
print(x.shape,y.shape)

(25945, 36) (25945, 1)


In [11]:
# one hot enconder
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()
y.shape

(25945, 5)

In [12]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

In [13]:
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(17383, 36) (8562, 36) (17383, 5) (8562, 5)


In [14]:
# standardscaler x data
sc_x = StandardScaler()
X_trainScaled = sc_x.fit_transform(X_train)
X_testScaled = sc_x.fit_transform(X_test)

In [15]:
clf = MLPClassifier(hidden_layer_sizes=(256,128,64,32),activation="relu",random_state=1).fit(X_trainScaled, y_train)
y_pred=clf.predict(X_testScaled)
print(clf.score(X_testScaled, y_test))

0.3937164213968699
